In [2]:
from data import data2

host = '192.168.70.1'
database = 'binance_1'
db = data2.data(host, database, True)
crypto_id = 1
start_time = '2024-01-01T00:00:00.000Z'
end_time = '2024-10-01T00:00:00.000Z'

df_prices = db.query_to_df(f"SELECT id, time_open, open, high, low, close, volume, trades, buy_volume, buy_quote_volume FROM price WHERE crypto_id = {crypto_id} AND time_open >= '{start_time}' AND time_open < '{end_time}' ORDER BY time_open")
display(df_prices.head())

df_features = db.query_to_df(f"SELECT * FROM price_feature WHERE price_id IN (SELECT id FROM price WHERE crypto_id = {crypto_id} AND time_open >= '{start_time}' AND time_open < '{end_time}') ORDER BY price_id")
display(df_features.head())

df_features = df_features.pivot(index='price_id', columns='feature_id', values='value')
display(df_features.head())

# merge prices and features
df = df_prices.merge(df_features, left_on='id', right_on='price_id', right_index=True)

display(df.head())

index = 2
for col in df.columns:
    if col != 'id' and col != 'time_open':
        df.insert(index, str(col) + '_prev', df[col].shift(1))
        index += 1

display(df.head()) 

# shift output by 1
df['buy_price'] = df['low'].shift(-1)
df['sell_price'] = df['high'].shift(-1)
display(df.head())


,id,time_open,open,high,low,close,volume,trades,buy_volume,buy_quote_volume
0,135951,2024-01-01 00:00:00,42283.58,42554.57,42261.02,42475.23,1271.68,47134,682.58,28957416.82
1,1747917,2024-01-01 01:00:00,42475.23,42775.00,42431.65,42613.56,1196.38,50396,712.32,30355645.35
2,135952,2024-01-01 02:00:00,42613.57,42638.41,42500.00,42581.10,685.22,29863,288.99,12301017.30
3,135957,2024-01-01 03:00:00,42581.09,42586.64,42230.08,42330.49,794.80,38620,356.37,15112996.38
4,135958,2024-01-01 04:00:00,42330.50,42399.99,42209.46,42399.99,715.42,36038,371.12,15703618.93


,id,price_id,feature_id,value
0,18046371,6703,571,10.43
1,49322982,6703,750,43765.11
2,49322983,6703,752,0.59
3,49322984,6703,751,452.90
4,49322985,6703,753,44090.30


feature_id,553,554,555,556,557,558,559,560,561,562,...,751,752,753,754,755,756,757,758,759,760
price_id,,,,,,,,,,,,,,,,,,,,,
6703,-98.93,12.27,35.33,-110.58,0.00,-65.19,38.28,0.38,-333.36,-10505.77,...,452.90,0.59,44090.30,0.02,0.02,NaN,1275.00,0.58,0.58,1275.00
6709,-185.79,11.52,35.90,-225.84,0.00,-83.00,41.18,-0.57,-299.43,-3720.71,...,360.82,0.61,44026.34,0.02,0.03,NaN,-1647.75,-0.96,-0.96,-1647.75
6710,15.10,20.76,31.51,71.81,0.00,52.11,29.38,0.98,57.21,527.36,...,190.23,0.51,43028.99,0.02,0.02,NaN,-230.97,-0.30,-0.30,-230.97
7762,-133.85,23.00,11.67,-97.30,0.00,48.59,28.94,-0.29,62.26,2991.61,...,261.04,0.48,43028.99,0.02,0.02,NaN,-1069.47,-0.91,-0.91,-1069.47
7838,-8.87,20.96,26.59,44.33,0.00,65.18,29.38,0.96,-18.88,-2742.49,...,142.77,0.49,43028.99,0.02,0.02,NaN,-247.79,-0.34,-0.34,-247.79


,id,time_open,open,high,low,close,volume,trades,buy_volume,buy_quote_volume,...,751,752,753,754,755,756,757,758,759,760
0,135951,2024-01-01 00:00:00,42283.58,42554.57,42261.02,42475.23,1271.68,47134,682.58,28957416.82,...,293.55,0.56,42477.50,0.02,0.02,NaN,584.27,0.46,0.46,584.27
1,1747917,2024-01-01 01:00:00,42475.23,42775.00,42431.65,42613.56,1196.38,50396,712.32,30355645.35,...,343.35,0.58,42477.50,0.02,0.02,NaN,71.33,0.06,0.06,71.33
2,135952,2024-01-01 02:00:00,42613.57,42638.41,42500.00,42581.10,685.22,29863,288.99,12301017.30,...,138.41,0.56,42477.50,0.02,0.02,NaN,117.78,0.17,0.17,117.78
3,135957,2024-01-01 03:00:00,42581.09,42586.64,42230.08,42330.49,794.80,38620,356.37,15112996.38,...,356.56,0.58,42477.50,0.02,0.02,NaN,-347.16,-0.44,-0.44,-347.16
4,135958,2024-01-01 04:00:00,42330.50,42399.99,42209.46,42399.99,715.42,36038,371.12,15703618.93,...,190.53,0.57,42477.50,0.02,0.02,NaN,715.42,1.00,1.00,715.42


,id,time_open,open_prev,high_prev,low_prev,close_prev,volume_prev,trades_prev,buy_volume_prev,buy_quote_volume_prev,...,751,752,753,754,755,756,757,758,759,760
0,135951,2024-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,293.55,0.56,42477.50,0.02,0.02,NaN,584.27,0.46,0.46,584.27
1,1747917,2024-01-01 01:00:00,42283.58,42554.57,42261.02,42475.23,1271.68,47134.00,682.58,28957416.82,...,343.35,0.58,42477.50,0.02,0.02,NaN,71.33,0.06,0.06,71.33
2,135952,2024-01-01 02:00:00,42475.23,42775.00,42431.65,42613.56,1196.38,50396.00,712.32,30355645.35,...,138.41,0.56,42477.50,0.02,0.02,NaN,117.78,0.17,0.17,117.78
3,135957,2024-01-01 03:00:00,42613.57,42638.41,42500.00,42581.10,685.22,29863.00,288.99,12301017.30,...,356.56,0.58,42477.50,0.02,0.02,NaN,-347.16,-0.44,-0.44,-347.16
4,135958,2024-01-01 04:00:00,42581.09,42586.64,42230.08,42330.49,794.80,38620.00,356.37,15112996.38,...,190.53,0.57,42477.50,0.02,0.02,NaN,715.42,1.00,1.00,715.42


,id,time_open,open_prev,high_prev,low_prev,close_prev,volume_prev,trades_prev,buy_volume_prev,buy_quote_volume_prev,...,753,754,755,756,757,758,759,760,buy_price,sell_price
0,135951,2024-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,42477.50,0.02,0.02,NaN,584.27,0.46,0.46,584.27,42431.65,42775.00
1,1747917,2024-01-01 01:00:00,42283.58,42554.57,42261.02,42475.23,1271.68,47134.00,682.58,28957416.82,...,42477.50,0.02,0.02,NaN,71.33,0.06,0.06,71.33,42500.00,42638.41
2,135952,2024-01-01 02:00:00,42475.23,42775.00,42431.65,42613.56,1196.38,50396.00,712.32,30355645.35,...,42477.50,0.02,0.02,NaN,117.78,0.17,0.17,117.78,42230.08,42586.64
3,135957,2024-01-01 03:00:00,42613.57,42638.41,42500.00,42581.10,685.22,29863.00,288.99,12301017.30,...,42477.50,0.02,0.02,NaN,-347.16,-0.44,-0.44,-347.16,42209.46,42399.99
4,135958,2024-01-01 04:00:00,42581.09,42586.64,42230.08,42330.49,794.80,38620.00,356.37,15112996.38,...,42477.50,0.02,0.02,NaN,715.42,1.00,1.00,715.42,42180.77,42406.00


In [1]:
from pycaret.regression import *
s = setup(df, target = 'buy_price', session_id = 123)

ModuleNotFoundError: No module named 'pycaret'